For another analysis I found that some of the user-ids are no Integers, this seems to be a data quality issue. In this notebook we try to get some statistics on this.

In [1]:
import commands as cmd

root_path = cmd.getoutput('pwd')[:cmd.getoutput('pwd').find('/databeers/')] + '/databeers'
data_path = root_path + "/data/"

In [2]:
import numpy as np
import pandas as pd
import sys
sys.path.append(root_path)

from src.read_write_utils import *

In [3]:
user_paths = [x[x.rfind('users_'):] for x in cmd.getoutput('ls %s/users_*'%data_path).split('\n')]

In [4]:
def check_integer(df, colname):
    """
    check in column of pandas dataframe how many entries are integers, non-integers and NA-values.
    """
    vc = df[colname].str.match("[0-9]+").value_counts()
    return vc.get(True, 0), vc.get(False, 0), len(df[colname]) - df[colname].count()


def check_id_cols(path):
    """
    check in a path if the id-column is correct. 
    Returns pd.DataFrame with statistics about correct, incorrect and NA-ids.
    """
    df_tmp = load_df(data_path + path)[["id", "id_str"]]
        
    id_corr, id_incorr, id_na = check_integer(df_tmp, "id")
    id_str_corr, id_str_incorr, id_str_na = check_integer(df_tmp, "id_str")
    
    return pd.DataFrame([[path, id_corr, id_incorr, id_na, id_str_corr, id_str_incorr, id_str_na]],
                         columns=['path', 'id_corr', 'id_incorr', 'id_na',
                                  'id_str_corr', 'id_str_incorr', 'id_str_na'])

In [5]:
check_id_cols('users_10060800.csv')

,path,id_corr,id_incorr,id_na,id_str_corr,id_str_incorr,id_str_na
0,users_10060800.csv,53,0,0,53,0,0


In [6]:
path_sample = np.random.choice(user_paths, size=1000, replace=False)

In [7]:
for idx, path in enumerate(path_sample.tolist()):
    if idx == 0:
        stats_df = check_id_cols(path)
    else:
        if idx%50 == 0:
            print 'progress', idx
        stats_df = stats_df.append(check_id_cols(path))

progress 50
progress 100
progress 150
progress 200
progress 250
progress 300
progress 350
progress 400
progress 450
progress 500
progress 550
progress 600
progress 650
progress 700
progress 750
progress 800
progress 850
progress 900
progress 950


In [8]:
stats_df["perc_incorr_id"] = stats_df["id_incorr"] / (stats_df["id_incorr"] + stats_df["id_corr"])
stats_df["perc_incorr_str_id"] = stats_df["id_str_incorr"] / (stats_df["id_str_incorr"] + stats_df["id_str_corr"])

In [9]:
print 'maximum incorrect id string', max(stats_df["perc_incorr_str_id"])
print 'maximum incorrect id int', max(stats_df["perc_incorr_id"])

maximum incorrect id string 0.00597907324365
maximum incorrect id int 0.00548081714001


In [10]:
for col in ["id_{}corr", "id_{}incorr" ,"id_{}na"]:
    print 'id == str_id in', col[col.rfind('}')+1:], all(stats_df[col.format("")] == stats_df[col.format("str_")].values)

id == str_id in corr False
id == str_id in incorr False
id == str_id in na True


In [11]:
clean_paths = [x[x.rfind('clean_users_'):] for x in cmd.getoutput('ls %s/daily_users/clean_users_*'%data_path).split('\n')]

In [12]:
for idx, path in enumerate(clean_paths):
    path = "daily_users/" + path
    if idx == 0:
        stats_clean_df = check_id_cols(path)
    else:
        if idx%10 == 0:
            print 'progress', idx
        stats_clean_df = stats_clean_df.append(check_id_cols(path))

progress 10
progress 20
progress 30
progress 40
progress 50
progress 60


In [13]:
stats_clean_df["perc_incorr_id"] = stats_clean_df["id_incorr"] / (stats_clean_df["id_incorr"] + stats_clean_df["id_corr"])
stats_clean_df["perc_incorr_str_id"] = stats_clean_df["id_str_incorr"] / (stats_clean_df["id_str_incorr"] + stats_clean_df["id_str_corr"])

In [14]:
print 'maximum incorrect id string in daily users', max(stats_clean_df["perc_incorr_str_id"])
print 'maximum incorrect id in daily users', max(stats_clean_df["perc_incorr_id"])

maximum incorrect id string in daily users 3.8731466993e-05
maximum incorrect id in daily users 0.0


The vast part of the user df seems to be ok, lets check the tweets as well!

In [15]:
tweet_paths = [x[x.rfind('tweets_'):] for x in cmd.getoutput('ls %s/tweets_*'%data_path).split('\n')]

In [16]:
def check_id_cols_tweets(path):
    df_tmp = load_df(data_path + path)[["user_id"]]
    
    id_corr, id_incorr, id_na = check_integer(df_tmp, "user_id")
    return pd.DataFrame([[path, id_corr, id_incorr, id_na]],
                         columns=['path', 'id_corr', 'id_incorr', 'id_na'])

In [17]:
for idx, path in enumerate(tweet_paths):
    if idx == 0:
        stats_tweets_df = check_id_cols_tweets(path)
    else:
        if idx%50 == 0:
            print idx
        stats_tweets_df = stats_tweets_df.append(check_id_cols_tweets(path))

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600


In [18]:
stats_tweets_df["perc_incorr_id"] = stats_tweets_df["id_incorr"] / (stats_tweets_df["id_incorr"] + stats_tweets_df["id_corr"])
print 'maximum percentage incorrect ids', round(max(stats_tweets_df["perc_incorr_id"])*100), '[%]'

maximum percentage incorrect ids 31.0 [%]


In [19]:
# hours with more than 1% incorrect
print len(stats_tweets_df[stats_tweets_df["perc_incorr_id"] > 0.01])
stats_tweets_df[stats_tweets_df["perc_incorr_id"] > 0.01]

43


,path,id_corr,id_incorr,id_na,perc_incorr_id
0,tweets_07081600.csv,6689,1672,0,0.199976
0,tweets_07081700.csv,8213,579,0,0.065855
0,tweets_07081800.csv,8926,429,0,0.045858
0,tweets_07081900.csv,9163,186,0,0.019895
0,tweets_07082000.csv,8966,175,0,0.019145
0,tweets_07082100.csv,9105,95,0,0.010326
0,tweets_07082200.csv,8573,3300,0,0.277942
0,tweets_07082300.csv,8506,820,0,0.087926
0,tweets_08080000.csv,8027,362,0,0.043152
0,tweets_08080100.csv,7604,214,0,0.027373


In [20]:
print 'min date', min([int(x[7:9]) + 100 * int(x[9:11]) for x in tweet_paths])
print 'max date', max([int(x[7:9]) + 100 * int(x[9:11]) for x in tweet_paths])

min date 609
max date 829


The tweet data also seems kind of alright, however there is a problem for some dates 27.07, 07.08, 08.08, 15.08